In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [65]:
#from https://gist.github.com/Maarten-vd-Sande/f751c1a456ce14d1d91e86846b31ae1f
#cross-checked against outputs from deseq2 implementation in R 

def deseq2_norm(_data : pd.DataFrame) -> pd.DataFrame:
    """
    Normalize a dataframe with the median of ratios method 
    from DESeq2. 
    
    See for an **excellent** explanation statquest (as usual)
    https://www.youtube.com/watch?v=UFB993xufUU
    
    input data (as a pandas dataframe), e.g.:
    
            sample1    sample2    sample3
    gene1   0.00000    10.0000    4.00000
    gene2   2.00000    6.00000    12.0000
    gene3   33.5000    55.0000    200.000
    
    normalized output:
            sample1    sample2    sample3
    gene1   0.00000    10.6444    1.57882
    gene2   4.76032    6.38664    4.73646
    gene3   78.5453    58.5442    78.9410
    """
    # step 1: log normalize
    log_data = np.log(_data)

    # step 2: average rows
    row_avg = np.mean(log_data, axis=1)
    
    # step 3: filter rows with zeros
    rows_no_zeros = row_avg[row_avg != -np.inf].index
    
    # step 4: subtract avg log counts from log counts
    ratios = log_data.loc[rows_no_zeros].subtract(row_avg.loc[rows_no_zeros], axis=0)
    
    # step 5: calculate median of ratios
    medians = ratios.median(axis=0)
    
    # step 6: median -> base number
    scaling_factors = np.e ** medians
    
    # step 7: normalize!
    normalized_data = _data / scaling_factors
    return normalized_data

In [66]:
files = glob.glob('data/*.xlsx')

## 1. Extract counts data from given files & save in standard format

In [132]:
file = files[3] 

In [133]:
file_prep = file[:-12]

In [134]:
file_prep

'data/TIF3_2nd_'

#### find sheets for gene / spliced / unspliced count data

In [135]:
xls = pd.ExcelFile(file)

In [136]:
data = pd.read_excel(xls, None)

In [137]:
data.keys()

dict_keys(['mapped to genes', 'mapped to unspliced fixed', 'mapped to spliced', 'conclusions using Deseq norm', 'conclusions using TPM'])

In [138]:
gene_df = data['mapped to genes']
unspliced_df = data['mapped to unspliced fixed']
spliced_df = data['mapped to spliced']

dfs = {
    "gene": gene_df,
    "unspliced": unspliced_df,
    "spliced": spliced_df
}

#### clean and save data for gene / unspliced / spliced

In [141]:
for key, df in dfs.items():
    #get desired columns and rows
    df = df.iloc[0:16869, [0, 6, 7, 8, 9, 10, 11]]
    
    filename = file_prep + key + "_data.csv"
    
    print(list(df))
    
    df.to_csv(filename, sep="\t", index=False)

['Geneid', 'TIF3_noR-noD-1 against unspliced', 'TIF3_noR-noD-2 against unspliced', 'TIF3_noR-noD-3 against unspliced', 'TIF3_noR-wD-1 against unspliced', 'TIF3_noR-wD-2 against unspliced', 'TIF3_noR-wD-3 against unspliced']
['Geneid', 'TIF3_noR-noD-1 against unspliced', 'TIF3_noR-noD-2 against unspliced', 'TIF3_noR-noD-3 against unspliced', 'TIF3_noR-wD-1 against unspliced', 'TIF3_noR-wD-2 against unspliced', 'TIF3_noR-wD-3 against unspliced']
['Geneid', 'TIF3_noR-noD-1 against unspliced', 'TIF3_noR-noD-2 against unspliced', 'TIF3_noR-noD-3 against unspliced', 'TIF3_noR-wD-1 against unspliced', 'TIF3_noR-wD-2 against unspliced', 'TIF3_noR-wD-3 against unspliced']


## 2. Perform Deseq2 median of ratios on all datasets

In [229]:
folders = glob.glob("data/*")

In [231]:
for folder in folders:
    files = glob.glob(folder + "/*.csv")

    #read dfs from files
    dfs = {}
    for file in files:
        df = pd.read_csv(file, sep='\t')
        df.set_index("Geneid", inplace=True)

        if "gene" in file:
            df.rename(columns=lambda x: x.split(" ")[0] + "_against_gene", inplace=True)
            dfs['gene'] = df
        elif "unspliced" in file:
            df.rename(columns=lambda x: x.split(" ")[0] + "_against_unspliced", inplace=True)
            dfs['unspliced'] = df
        elif "spliced" in file:
            df.rename(columns=lambda x: x.split(" ")[0] + "_against_spliced", inplace=True)
            dfs['spliced'] = df


    #perform median of ratios
    mor = {}
    for key, df in dfs.items():

        normed = deseq2_norm(df)
        normed.rename(columns=lambda x: x + "_mor", inplace=True)
        mor[key] = normed


    #add one to all mor
    mor_add_1 = {}
    for key, df in mor.items():

        add_1 = df + 1
        add_1.rename(columns=lambda x: x + "_add_1", inplace=True)
        mor_add_1[key] = add_1


    #normalize to gene
    spliced_normed_to_gene = mor_add_1["spliced"].values / mor_add_1["gene"].values
    spliced_normed_to_gene = pd.DataFrame(spliced_normed_to_gene, index = mor_add_1["spliced"].index,
                                         columns = mor_add_1['spliced'].columns)
    spliced_normed_to_gene.rename(columns=lambda x: x + "_normed_to_gene", inplace=True)

    unspliced_normed_to_gene = mor_add_1["unspliced"].values / mor_add_1["gene"].values
    unspliced_normed_to_gene = pd.DataFrame(unspliced_normed_to_gene, index = mor_add_1["unspliced"].index,
                                         columns = mor_add_1['unspliced'].columns)
    unspliced_normed_to_gene.rename(columns=lambda x: x + "_normed_to_gene", inplace=True)

    #get integer values
    gene_avg = mor_add_1["gene"].mean(axis=None) #average of all gene data

    spliced_normed_int = np.ceil(spliced_normed_to_gene * gene_avg)
    spliced_normed_int.rename(columns=lambda x: x + "_int", inplace=True)

    unspliced_normed_int = np.ceil(unspliced_normed_to_gene * gene_avg)
    unspliced_normed_int.rename(columns=lambda x: x + "_int", inplace=True)


    #write to xlsx file
    dataname = folder.split("/")[1]
    filename = dataname + "_MOR_summary.xlsx"

    #sheet 1: gene [counts, mor, mor+1]
    gene_sheet = pd.concat([dfs['gene'], mor['gene'], mor_add_1['gene']], axis=1)
    gene_sheetname = dataname + "_genes"

    #sheet 2: unspliced [counts, mor, mor+1, mor+1 normed to gene, mor+1 normed to gene int]
    unspliced_sheet = pd.concat([dfs['unspliced'],
                                mor['unspliced'],
                                mor_add_1['unspliced'],
                                unspliced_normed_to_gene,
                                unspliced_normed_int],
                                axis = 1)
    unspliced_sheetname = dataname + "_unspliced"

    #sheet 3: spliced   [counts, mor, mor+1, mor+1 normed to gene, mor+1 normed to gene int]
    spliced_sheet = pd.concat([dfs['spliced'],
                                mor['spliced'],
                                mor_add_1['spliced'],
                                spliced_normed_to_gene,
                                spliced_normed_int],
                                axis = 1)
    spliced_sheetname = dataname + "_spliced"

    with pd.ExcelWriter(filename) as writer:

        # use to_excel function and specify the sheet_name and index 
        # to store the dataframe in specified sheet
        gene_sheet.to_excel(writer, sheet_name=gene_sheetname)
        unspliced_sheet.to_excel(writer, sheet_name=unspliced_sheetname)
        spliced_sheet.to_excel(writer, sheet_name=spliced_sheetname)

/Users/larkpie/mambaforge/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/larkpie/mambaforge/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/larkpie/mambaforge/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/larkpie/mambaforge/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/larkpie/mambaforge/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/larkpie/mambaforge/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWa